## Подготовка данных

In [10]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
local_crs = 4326
example_data_path = "./VRP/exp_data/"

In [11]:
dodo = gpd.read_file(os.path.join(example_data_path, "Данные о доходах додо пиццы.geojson")).to_crs(local_crs)
blocks = gpd.read_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet")).to_crs(local_crs)
accessibility_matrix = pd.read_pickle(os.path.join(example_data_path, "matrix_mon_0900.pickle"))
demands = pd.read_csv(os.path.join(example_data_path, "second_scenario.csv"))
demands.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

In [12]:
blocks['centroid'] = blocks['geometry'].centroid
blocks.geometry = blocks['centroid']
blocks.drop(columns=['centroid', 'landuse'], inplace=True)
# Получение 40 случайных строк из исходного GeoDataFrame
blocks_40 = blocks.sample(n=40, random_state=1)  # Используйте другой random_state для другой выборки
# Создание нового GeoDataFrame на основе случайных строк
blocks_40 = gpd.GeoDataFrame(blocks_40, geometry='geometry', crs=blocks_40.crs)
blocks_40.reset_index(drop=True, inplace=True)
blocks_40['vrp_id'] = blocks_40.index
blocks_40.head()

C:\Users\skitl\AppData\Local\Temp\ipykernel_404\2260754542.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks['centroid'] = blocks['geometry'].centroid


,geometry,id,vrp_id
0,POINT (30.23725 59.94681),91,0
1,POINT (30.23616 59.93375),443,1
2,POINT (30.27768 59.94933),776,2
3,POINT (30.23291 59.95444),482,3
4,POINT (30.27220 59.93560),1073,4


In [13]:
# Функция для извлечения координат из геометрии
def extract_coordinates(row):
    return row['geometry'].x, row['geometry'].y

# Применение функции к каждой строке GeoDataFrame
blocks_40['coordinates'] = blocks_40.apply(extract_coordinates, axis=1)

# Преобразование координат в массив numpy
coordinates_array = np.array(blocks_40['coordinates'].tolist())

coordinates_array

array([[30.23725223, 59.94681097],
       [30.23615802, 59.93375286],
       [30.277682  , 59.94933284],
       [30.23291334, 59.95444208],
       [30.27220029, 59.9355989 ],
       [30.2920856 , 59.94028113],
       [30.24740838, 59.95465574],
       [30.2465466 , 59.94967585],
       [30.27443905, 59.93945694],
       [30.28112209, 59.94055325],
       [30.26709624, 59.93848723],
       [30.26132438, 59.93312537],
       [30.2253302 , 59.9326824 ],
       [30.20965571, 59.95089336],
       [30.24014745, 59.95016267],
       [30.27370056, 59.93642756],
       [30.23820194, 59.93056522],
       [30.22145889, 59.93357544],
       [30.26736085, 59.95338766],
       [30.28285029, 59.9481095 ],
       [30.2742119 , 59.93677489],
       [30.24658721, 59.93650089],
       [30.27102015, 59.93064191],
       [30.2679881 , 59.93489392],
       [30.25979894, 59.94808359],
       [30.23831776, 59.94709691],
       [30.26626537, 59.94770951],
       [30.22902667, 59.9585606 ],
       [30.22322408,

In [14]:
# Получение списка id кварталов из GeoDataFrame
quarters_ids = blocks_40['id'].tolist()

# Фильтрация матрицы связанности по кварталам из GeoDataFrame
filtered_matrix_df = accessibility_matrix.loc[quarters_ids, quarters_ids]

# Вывод отфильтрованной матрицы связанности
filtered_matrix_df.head()

id,91,443,776,482,1073,810,556,547,215,1093,...,767,744,1126,1118,680,201,1070,977,303,248
id,,,,,,,,,,,,,,,,,,,,,
91,0.0,6.1,11.8,7.9,12.4,15.9,9.5,7.7,12.6,12.7,...,9.6,14.6,14.8,14.6,12.3,12.4,10.3,9.2,9.8,14.8
443,5.0,0.0,13.0,9.9,10.1,15.7,11.5,9.7,12.4,12.5,...,10.8,13.6,14.7,13.6,10.0,13.6,11.5,3.6,11.0,15.4
776,12.2,15.0,0.0,14.5,7.9,6.0,11.5,9.9,5.5,5.6,...,2.1,7.3,4.9,4.9,7.8,6.2,5.2,15.1,7.4,4.3
482,5.3,10.2,14.5,0.0,16.3,19.8,7.5,5.7,16.5,16.6,...,13.5,18.5,18.7,18.5,16.2,11.4,10.9,13.6,9.8,18.7
1073,11.5,11.2,8.7,15.1,0.0,5.6,13.7,10.5,3.4,3.2,...,7.5,3.5,4.6,3.5,0.1,10.3,8.2,7.8,8.0,5.9


In [15]:
array_matrix = filtered_matrix_df.values#.astype(int)
# array_result = array_result.reshape(-1, 1)
array_matrix

array([[ 0. ,  6.1, 11.8, ...,  9.2,  9.8, 14.8],
       [ 5. ,  0. , 13. , ...,  3.6, 11. , 15.4],
       [12.2, 15. ,  0. , ..., 15.1,  7.4,  4.3],
       ...,
       [ 8.3,  3.4, 14.7, ...,  0. , 12.7, 14.2],
       [ 9.3, 12.1,  6.4, ..., 13.1,  0. , 10.6],
       [14.3, 14.9,  4.2, ..., 13.9,  9.5,  0. ]])

In [60]:
df

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 19:00:00,88,62,42,41,80,31,79,0,66,63,...,77,80,86,67,75,65,104,72,41,50
2023-09-30 20:00:00,76,68,38,37,79,29,73,0,61,59,...,68,70,80,62,76,61,88,68,32,47
2023-09-30 21:00:00,65,46,30,33,69,19,56,0,54,54,...,44,61,80,59,74,56,56,59,16,44


In [16]:
import pandas as pd

# Предположим, что ваш DataFrame называется orders_df
df = demands
# Преобразуем столбец 'date' в datetime
df['date'] = pd.to_datetime(df['date'])

# Устанавливаем 'date' в качестве индекса
df.set_index('date', inplace=True)

# Агрегируем данные по дням и суммируем заказы
daily_orders = df.resample('D').sum()

# Выводим новый датафрейм
daily_orders


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,653,0,0,0,510,175,498,597,385,182,...,574,436,806,568,811,547,705,0,0,0
2022-01-02,624,0,0,0,485,167,474,569,367,171,...,546,413,768,539,773,520,670,0,0,0
2022-01-03,343,0,0,0,267,94,262,312,202,94,...,303,229,423,297,426,286,370,0,0,0
2022-01-04,312,0,0,0,243,84,235,287,183,86,...,276,207,384,269,386,262,336,0,0,0
2022-01-05,342,0,0,0,266,91,262,312,202,95,...,301,228,423,295,424,285,370,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-26,465,289,240,249,407,153,407,0,333,337,...,379,484,502,357,417,318,556,414,200,299
2023-09-27,512,320,262,275,447,167,445,0,365,368,...,416,533,551,395,458,351,609,457,221,329
2023-09-28,558,347,286,298,488,181,484,0,399,402,...,454,582,601,427,504,383,665,497,240,356


In [18]:
# Функция, которая принимает дату и возвращает список заказов на эту дату
def get_orders_by_date(date, array):
    if array == True:
        try:
            orders = daily_orders.loc[date]
            return np.array(orders)
        except KeyError:
            return np.array([])  # Если дата не найдена, возвращаем пустой список
    else:
        try:
            orders = daily_orders.loc[date]
            return orders.tolist()
        except KeyError:
            return []  # Если дата не найдена, возвращаем пустой список
# Пример использованияЫ
input_date = '2023-09-30'  # Замените на нужную дату
result = get_orders_by_date(input_date, array = False)
print(f"Заказы на {input_date}: {result}")

Заказы на 2023-09-30: [977, 609, 500, 523, 855, 318, 849, 0, 700, 702, 809, 850, 607, 533, 590, 716, 891, 904, 354, 1014, 932, 969, 856, 998, 1249, 1221, 1021, 666, 1093, 1089, 793, 1018, 1054, 750, 880, 669, 1163, 869, 420, 626]


## OR-Tools

In [63]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection


In [64]:
"""Capacited Vehicles Routing Problem (CVRP)."""

start_node = '91'
# Задать доступное количество транспортных средств (фиксированное)
num_vehicles = 3

# Для каждой даты решить задачу CVRP
input_date = '2023-08-30'  # Замените на нужную дату
demand = get_orders_by_date(input_date, array = False)
num_vehicles = 3
time_matrix = array_matrix
time_matrix = time_matrix.astype(int)
total_time = 0

In [68]:

def create_data_model():
    """Stores the data for the problem."""
    demand[0] = 0  
    data = {}
    data["distance_matrix"] = time_matrix
    data["demands"] = demand
    data["vehicle_capacities"] = [15000, 15000, 15000, 15000]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )
  
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == "__main__":
    main()

Objective: 126
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 3:
 0 Load(0) ->  14 Load(224) ->  29 Load(793) ->  6 Load(1165) ->  35 Load(1529) ->  18 Load(1817) ->  13 Load(2074) ->  27 Load(2373) ->  3 Load(2652) ->  25 Load(3202) ->  7 Load(3202) ->  38 Load(3388) ->  24 Load(3918) ->  36 Load(4443) ->  26 Load(4903) ->  30 Load(5255) ->  19 Load(5741) ->  2 Load(5998) ->  39 Load(6311) ->  5 Load(6459) ->  32 Load(6935) ->  33 Load(7273) ->  31 Load(7779) ->  9 Load(8129) ->  8 Load(8426) ->  10 Load(8790) ->  34 Load(9188) ->  20 Load(9617) ->  15 Load(9935) ->  4 Load(10319) ->  22 Load(10765) ->  23 Load(11214) ->  11 Load(11602) ->  21 Load(12015) ->  37 Load(12370) ->  16 Load(12739) ->  1 Load(12991) ->  12 Load(13293) ->  17 

## GNN

In [24]:
import torch
import os
import numpy as np
from torch_geometric.data import Data, DataLoader
from VRP.creat_vrp import reward1,creat_instance
from VRP.VRP_Actor import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [25]:
def cal_cost(routes):
    # Вычисление длины каждого маршрута
    total_length = 0.0
    for route in routes:
        route_length = sum(array_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        route_length += array_matrix[0][route[0]]
        route_length += array_matrix[route[-1]][0]
        total_length += route_length
    return total_length

In [59]:
datas = []
n_nodes = 40
actor = 51
x=0 # выбираем набор данных

node_ = coordinates_array
input_date = '2023-08-30'  # Замените на нужную дату
demand_ = get_orders_by_date(input_date, array = True)
demand_[demand_ == 0] = 500
demand_[0] = 0
capcity_ = np.array([5000.]) #настройка вместимости транспорта

# node_ = node_.reshape(-1, n_nodes, 2)
node_,demand_=node_.reshape(-1,n_nodes,2),demand_.reshape(-1,n_nodes)

# data_size = node_.shape[0]
np_matrix = array_matrix
np_matrix = np_matrix.astype(int)
np_matrix = np_matrix.reshape(-1, 1)


edges_index = []
for i in range(n_nodes):
    for j in range(n_nodes):
        edges_index.append([i, j])
edges_index = torch.LongTensor(edges_index)
edges_index = edges_index.transpose(dim0=0, dim1=1)

agent = Model(3, 128, 1, 16, conv_laysers=4).to(device)
agent.to(device)
folder = 'trained'
filepath = os.path.join(folder, '%s' % actor)
if os.path.exists(filepath):
        path1 = os.path.join(filepath, 'actor.pt')
        agent.load_state_dict(torch.load(path1, device))

datas_ = []
batch_size1 = 12 # sampling batch_size
for y in range(120):
    data = Data(x=torch.from_numpy(node_[x]).float(), 
                edge_index=edges_index,
                edge_attr=torch.from_numpy(np_matrix).float(),
                demand=torch.tensor(demand_[x]).unsqueeze(-1).float(),
                capcity=torch.tensor(capcity_[0]).unsqueeze(-1).float())
    datas_.append(data)
dl = DataLoader(datas_, batch_size=batch_size1)

min_tour=[]
min_cost=1000
T=2.5#Temperature hyperparameters

for batch in dl:
    costs = []
    with torch.no_grad():
        batch.to(device)
        tour1, _ = agent(batch, n_nodes * 2, False, T)
        for tour in tour1:
            routes_1 = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
            cost = cal_cost(routes_1)
            costs.append(cost)
        id = np.array(costs).argmin()
        m_cost=np.array(costs).min()
        if m_cost<min_cost:
            min_cost=m_cost
            min_tour=tour1[id]

tour=min_tour.unsqueeze(-2)
print('Problem:VRP''%s' % n_nodes,'/ Average distance:', min_cost)
for i, (data, tour) in enumerate(zip(dl, tour)):
    routes = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)

print('_______________')

depot = data.x[0].cpu().numpy()
locs = data.x[1:].cpu().numpy()
demands = data.demand.cpu().numpy()
demands=demands[1:]
total = 0
for veh_number, r in enumerate(routes):
    route_demands = demands[r - 1]
    coords = locs[r - 1, :]
    xs, ys = coords.transpose()
    total_route_demand = sum(route_demands)
    total += total_route_demand
    print(f'Маршрут {veh_number}, Груз на маршруте - {total_route_demand}, Количество посещаемых узлов - {len(r)}')
print('Total demands''%s' % total)

Problem:VRP40 / Average distance: 391.0
[array([36, 25, 14, 27, 31, 24, 19,  4, 22, 39, 29], dtype=int64), array([ 6, 11, 32,  8, 23, 20, 17, 28, 35, 30, 33,  7,  5], dtype=int64), array([12, 34,  9, 10, 13,  3, 15, 26,  2, 18, 37, 21,  1, 16, 38],
      dtype=int64)]
_______________
Маршрут 0, Груз на маршруте - [4832.], Количество посещаемых узлов - 11
Маршрут 1, Груз на маршруте - [4969.], Количество посещаемых узлов - 13
Маршрут 2, Груз на маршруте - [4848.], Количество посещаемых узлов - 15
Total demands[14649.]
